**Dashpot** (Damper)
    - Dissipating Spring Force
    - Damping or loss of energy over time
    - e.g. : air drag, mechanical damping, etc
    - Applies when there is a relative velocity between the particles in a spring system

![](https://i.ibb.co/2j72q9L/Screenshot-from-2019-03-08-16-55-11.png)

**Explicit vs Implicit Euler**

**Explicit**
    - The next position completely depends on the parameters of the current timestep
    - Can be problematic if the time step is taken large
        - The system can blow up as the current state may change drastically at a large timestep
        
**Implicit**
    - It depends on the parameters that are evaluated at the end of the timestep instead of the beginning
    - x = f(x)
        - Unknown is the function of its own (as it depends on its own value)
    - **Newton's Method** (A quadratic convergence method)
    

**Newton's method**
- For linear approximation, we use Taylor's function
- Disadv:
    - Can never converge if we start with a random guess that is too far from the original root

![](https://i.ibb.co/0D1DjC0/Screenshot-from-2019-03-11-17-44-52.png)
![](https://i.ibb.co/5K4m7xX/Screenshot-from-2019-03-11-17-53-02.png)

## Implicit Euler

- Can handle changes for large timesteps and maintain the system to be stable
- Costlier than explicit euler at each timestep, but is often found to be productive

- Position, velocity and state when applying Implicit Euler:

![](https://i.ibb.co/qrSZSKX/Screenshot-from-2019-03-26-16-40-53.png)

- Implicit euler and Explicit euler are time-reversed version of each other
- You can retrieve one using the other by following a time-reversed version of the former

### Integrators
- Some integrators are the time-reversed version of itself
- Example:
    ![](https://i.ibb.co/b3KZLSm/Screenshot-from-2019-04-04-17-15-43.png)

### Jacobian Matrix
- A matrix of all first-order partial derivatives of a vector-valued function
- A position (q) matrix, A force (F) matrix, and a Jacobian (J) matrix

# Week 4

### 1. Detecting collisions between two balls

- Detecting a collision
    - Instantaneous approach
    - Continuous duration approach


![](https://i.ibb.co/3dv73Gd/Screenshot-from-2019-04-04-17-43-31.png)
    
### 2. Detecting collision between a ball and a wall


![](https://i.ibb.co/WFyH21W/Screenshot-from-2019-04-04-17-49-07.png)
![](https://i.ibb.co/6NxDGbq/Screenshot-from-2019-04-04-17-51-42.png)


### 3. Detecting collision between a ball and an edge

- For detecting collision between a ball and an edge, there are two conditions according to the value of alpha
- When alpha = 0 to 1, the ball lies within the vertical range of the edge
- When aplha < 0, we assume that the alpha = 0 and when aplha > 1 , we assume alpha = 1
    - Since the position of X(alpha) gets changed for aplha not in rangfe (0, 1), the formula for detecting the collision remains the same
    - **|| X1 - Xalpha || < r1 + r** (Overlapping condition)
    - X1 ==> position of the center of the ball
    - Xalpha ==> position of alpha on the line
    - r1 ==> radius of the ball
    - r ==> radius of the edge convex

- To determine the direction of the movement of the ball:
    - (X1 - Xalpha) . (V1 - V2) > 0   **(moving away or receding)**

![](https://i.ibb.co/ZH8Lrbp/Screenshot-from-2019-04-04-18-01-36.png)


![](https://i.ibb.co/2qhBKpb/Screenshot-from-2019-04-08-12-38-31.png)

### Instantaneous Collision Response

- When two balls approach eachother, there needs to be some formula to calculate the behavior of the collision so that we can react appropriately to the collision instead of the balls passing eachother
    - We use conservation of momentum and conservation of energy to determine the value of **alpha** and hence the **Impulse** of the collision
    - The impulse of the collision only acts on the direction of the velocities and it does not affect the perpendicular components of the velocities of the balls
    
    
![](https://i.ibb.co/rGQ7nJb/Screenshot-from-2019-04-08-12-53-55.png)

![](https://i.ibb.co/VHNrZ9d/Screenshot-from-2019-04-08-12-54-09.png)

### Coefficient of Restituition

- When an object is dropped from a height, it rebounces to back to a certain fraction of the  original height
- This fraction is the **coefficient of restitution**
- COR is between [0, 1]
- Defines how bouncy an object is

- When COR = 0:
    - Maximally dissipative collision (All kinetic energy is lost)
    - The impulse in this case is **I/2**
        - This means that **I/2** is the least impulse that is required to stop the balls from colliding (passing through eachother)

### Summary of the impulse response for two colliding bodies

![](https://i.ibb.co/PrGkk2z/Screenshot-from-2019-04-08-14-26-01.png)

### Impulse Response with fixed particles

- When a moving object collides with a fixed particle with comparitively infinite mass (e.g. a wall), the velocity of the fixed particle does not change
- So, to represent a fixed particle in C++, we use the standard **infinity** expression and assign it to the mass of the fixed particle

### Penalty Method

- Instead of treating collisions with **impulse** (i.e. changing the velocity instantaneously after collision), we can also gradually change the velocity of particles through a time interval. This is called **penalty method**


#### 1. Interior penalty
- We exert some force (instead of impulses) on both the particles when they start touching
- This does not change the velocities of the particles instantaneously, but applies acceleration (-ve genrally) that gradually move the bodies apart from each other after touching/passing through eachother for some instant

#### 2. Exterior penalty
- Forces start to act before the bodies actually touch eachother
- Generally creating a bigger radius over the ball (which when touch, the forces get applied)


![](https://i.ibb.co/R0Gj5gJ/Screenshot-from-2019-04-08-15-01-46.png)

- In penalty method, we assume that there is a spring-like force that exists between the two approaching bodies
- The energy/force increases as the distance between the approaching bodies decreases
    - This makes up to a collision and changes the velocity gradually
    
#### For interior penalty
- The more the bodies overlap, the greater is the force
- The force starts acting only when the bodies start touching eachother
![](https://i.ibb.co/Y8X1TYr/Screenshot-from-2019-04-08-15-23-21.png)


#### For exterior penalty
- The forces start acting when the bigger radii start touching eachother
- This accelerates the bodies even before they touch eachother

### Contrasting Penalty and Impulse approach

- Advantages of Penalty
    - Its an energy that we code up. We can just take in the derivative of the energy and compute the forces acting on the bodies
    - We can use an integrator (e.g. implicit integrator) to compute the forces
- Disadvantages
    - **k & T** (only for exterior) are self-assumed variables and cannot be explicitly calculated. We just assume them by experimentation. These parameters are set for numerical convenience
    - In case of impulse approach, the parameter **COR** can be calculated using the available metrics 
    
    
- When the objects are approaching too rapidly, we use **Impulse approach**
- For objects nearly resting on eachother (e.g. boxes on top of eachother with gravity), we use **Penalty approach**
    - If we apply the impulsive approach in this case, we may find a bit of **jittering** between the bodies as the impulse acts on each instant trying to respond to the collision
    